In [15]:
import numpy as np
import tensorflow as tf

In [16]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [17]:
tf.__version__

'2.1.0'

In [18]:
hub.__version__

'0.7.0'

In [31]:
tfds.list_builders()

['abstract_reasoning',
 'aflw2k3d',
 'amazon_us_reviews',
 'bair_robot_pushing_small',
 'bigearthnet',
 'binarized_mnist',
 'binary_alpha_digits',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_corrupted',
 'clevr',
 'cnn_dailymail',
 'coco',
 'coco2014',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'diabetic_retinopathy_detection',
 'downsampled_imagenet',
 'dsprites',
 'dtd',
 'dummy_dataset_shared_generator',
 'dummy_mnist',
 'emnist',
 'eurosat',
 'fashion_mnist',
 'flores',
 'food101',
 'gap',
 'glue',
 'groove',
 'higgs',
 'horses_or_humans',
 'image_label_folder',
 'imagenet2012',
 'imagenet2012_corrupted',
 'imdb_reviews',
 'iris',
 'kitti',
 'kmnist',
 'lfw',
 'lm1b',
 'lsun',
 'mnist',
 'mnist_corrupted',
 'moving_mnist',
 'multi_nli',
 'nsynth',
 'omniglot',

In [46]:
(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(tfds.Split.TRAIN.subsplit([6, 4]), tfds.Split.TEST), 
    as_supervised=True)

In [49]:
type(train_data)

tensorflow.python.data.ops.dataset_ops._OptionsDataset

In [67]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by

In [58]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0], dtype=int64)>

In [63]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch)

<tf.Tensor: shape=(10, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ],
       [ 3.9022908 , -5.336768  ,  4.6433706 , -2.322609  , -5.6409845 

In [68]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_3 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [69]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [70]:
model.fit(train_data.shuffle(10000).batch(512),
          epochs=20,
          validation_data=validation_data.batch(512),
          verbose=1)

Epoch 1/20
30/30 [==============================].7283 - accuracy: 0.50 - 3s 1s/step - loss: 0.7345 - accuracy: 0.50 - 3s 993ms/step - loss: 0.7359 - accuracy: 0.492 - 3s 772ms/step - loss: 0.7339 - accuracy: 0.481 - 3s 642ms/step - loss: 0.7335 - accuracy: 0.482 - 3s 551ms/step - loss: 0.7325 - accuracy: 0.490 - 3s 487ms/step - loss: 0.7289 - accuracy: 0.496 - 3s 435ms/step - loss: 0.7252 - accuracy: 0.496 - 4s 396ms/step - loss: 0.7239 - accuracy: 0.498 - 4s 367ms/step - loss: 0.7220 - accuracy: 0.499 - 4s 343ms/step - loss: 0.7199 - accuracy: 0.499 - 4s 321ms/step - loss: 0.7186 - accuracy: 0.502 - 4s 303ms/step - loss: 0.7168 - accuracy: 0.502 - 4s 288ms/step - loss: 0.7153 - accuracy: 0.502 - 4s 275ms/step - loss: 0.7134 - accuracy: 0.502 - 4s 263ms/step - loss: 0.7124 - accuracy: 0.502 - 4s 253ms/step - loss: 0.7110 - accuracy: 0.502 - 4s 243ms/step - loss: 0.7098 - accuracy: 0.503 - 4s 235ms/step - loss: 0.7089 - accuracy: 0.503 - 5s 227ms/step - loss: 0.7076 - accuracy: 0.502 -

30/30 [==============================] - ETA: 38s - loss: 0.6410 - accuracy: 0.712 - ETA: 21s - loss: 0.6427 - accuracy: 0.719 - ETA: 14s - loss: 0.6392 - accuracy: 0.716 - ETA: 11s - loss: 0.6405 - accuracy: 0.721 - ETA: 9s - loss: 0.6362 - accuracy: 0.721 - ETA: 8s - loss: 0.6357 - accuracy: 0.72 - ETA: 7s - loss: 0.6377 - accuracy: 0.72 - ETA: 6s - loss: 0.6357 - accuracy: 0.72 - ETA: 5s - loss: 0.6343 - accuracy: 0.71 - ETA: 5s - loss: 0.6345 - accuracy: 0.71 - ETA: 4s - loss: 0.6343 - accuracy: 0.72 - ETA: 4s - loss: 0.6347 - accuracy: 0.72 - ETA: 4s - loss: 0.6352 - accuracy: 0.72 - ETA: 3s - loss: 0.6346 - accuracy: 0.72 - ETA: 3s - loss: 0.6356 - accuracy: 0.72 - ETA: 3s - loss: 0.6344 - accuracy: 0.72 - ETA: 2s - loss: 0.6343 - accuracy: 0.72 - ETA: 2s - loss: 0.6346 - accuracy: 0.72 - ETA: 2s - loss: 0.6342 - accuracy: 0.72 - ETA: 1s - loss: 0.6336 - accuracy: 0.72 - ETA: 1s - loss: 0.6333 - accuracy: 0.72 - ETA: 1s - loss: 0.6335 - accuracy: 0.72 - ETA: 1s - loss: 0.6331 - a

Epoch 12/20
30/30 [==============================] - ETA: 34s - loss: 0.5847 - accuracy: 0.861 - ETA: 18s - loss: 0.5838 - accuracy: 0.865 - ETA: 13s - loss: 0.5829 - accuracy: 0.863 - ETA: 10s - loss: 0.5793 - accuracy: 0.863 - ETA: 8s - loss: 0.5800 - accuracy: 0.860 - ETA: 7s - loss: 0.5823 - accuracy: 0.86 - ETA: 6s - loss: 0.5807 - accuracy: 0.85 - ETA: 6s - loss: 0.5788 - accuracy: 0.86 - ETA: 5s - loss: 0.5794 - accuracy: 0.85 - ETA: 5s - loss: 0.5790 - accuracy: 0.85 - ETA: 4s - loss: 0.5783 - accuracy: 0.85 - ETA: 4s - loss: 0.5777 - accuracy: 0.85 - ETA: 3s - loss: 0.5786 - accuracy: 0.85 - ETA: 3s - loss: 0.5780 - accuracy: 0.85 - ETA: 3s - loss: 0.5773 - accuracy: 0.85 - ETA: 2s - loss: 0.5773 - accuracy: 0.85 - ETA: 2s - loss: 0.5767 - accuracy: 0.85 - ETA: 2s - loss: 0.5773 - accuracy: 0.85 - ETA: 2s - loss: 0.5774 - accuracy: 0.85 - ETA: 1s - loss: 0.5771 - accuracy: 0.85 - ETA: 1s - loss: 0.5771 - accuracy: 0.85 - ETA: 1s - loss: 0.5771 - accuracy: 0.86 - ETA: 1s - loss

Epoch 18/20
30/30 [==============================] - ETA: 36s - loss: 0.5450 - accuracy: 0.902 - ETA: 19s - loss: 0.5556 - accuracy: 0.902 - ETA: 13s - loss: 0.5552 - accuracy: 0.903 - ETA: 10s - loss: 0.5508 - accuracy: 0.910 - ETA: 8s - loss: 0.5507 - accuracy: 0.907 - ETA: 7s - loss: 0.5513 - accuracy: 0.90 - ETA: 6s - loss: 0.5512 - accuracy: 0.90 - ETA: 5s - loss: 0.5531 - accuracy: 0.90 - ETA: 5s - loss: 0.5536 - accuracy: 0.90 - ETA: 4s - loss: 0.5549 - accuracy: 0.90 - ETA: 4s - loss: 0.5542 - accuracy: 0.90 - ETA: 3s - loss: 0.5536 - accuracy: 0.90 - ETA: 3s - loss: 0.5538 - accuracy: 0.90 - ETA: 3s - loss: 0.5541 - accuracy: 0.90 - ETA: 2s - loss: 0.5529 - accuracy: 0.90 - ETA: 2s - loss: 0.5526 - accuracy: 0.90 - ETA: 2s - loss: 0.5515 - accuracy: 0.90 - ETA: 2s - loss: 0.5514 - accuracy: 0.90 - ETA: 1s - loss: 0.5507 - accuracy: 0.90 - ETA: 1s - loss: 0.5507 - accuracy: 0.90 - ETA: 1s - loss: 0.5510 - accuracy: 0.90 - ETA: 1s - loss: 0.5514 - accuracy: 0.91 - ETA: 1s - loss

In [71]:
model.predict(['this a bad movie'])

array([[0.00691952]], dtype=float32)

In [72]:
model.predict(['this a worst movie'])

array([[0.00585464]], dtype=float32)

In [73]:
model.predict(['this is a good movie'])

array([[0.74486524]], dtype=float32)

In [75]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.574
accuracy: 0.848
